In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import sys
from datetime import datetime, date
import os
from IPython.display import clear_output

In [ ]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:96.0) Gecko/20100101 Firefox/96.0'}
FQDN = 'https://www.o*********.**'
responce = requests.get(FQDN)

print("request denied") if responce.status_code != 200 else print("ok")

In [ ]:
def get_html(url, params=None):
    r = requests.get(url, headers=header, params=params)
    return r

---

In [ ]:
page_url = 'https://www.o********.***'
response = get_html(page_url)
soup = BeautifulSoup(response.text, 'lxml')

fqdn_urls = [FQDN + e.a.get('href') for e in soup.find_all('tr') if 'http' not in e.a.get('href')]
titles = [e.a.text for e in soup.find_all('tr') if 'http' not in e.a.get('href')]
destination = [e.text.split('   ')[0].strip() for e in soup.find_all('td')][2:][::2]
source = [e.text for e in soup.find_all('div', class_='petitions-item-author')]
dop_urls = [FQDN + '/' + url for url in str([e for e in soup.find_all('div', id='petition-sign')][0]).split('"') if 'index2' in url]

In [ ]:
for url in dop_urls:
    response = get_html(url)
    soup = BeautifulSoup(response.text, 'lxml')
    dop_u = [FQDN + e.a.get('href') for e in soup.find_all('td') if e.a]
    dop_t = [e.a.text for e in soup.find_all('td') if e.a]
    dop_d = [e.text.split('   ')[0].strip() for e in soup.find_all('td')][::2]
    dop_s= [e.text for e in soup.find_all('div', class_='petitions-item-author')]
    fqdn_urls.extend(dop_u)
    titles.extend(dop_t)
    destination.extend(dop_d)
    source.extend(dop_s)

In [ ]:
content = []
subscribers = []
for url in fqdn_urls[:]:
    clear_output(wait=True)
    print(url)
    print(len(subscribers), len(content))
    response = get_html(url)
    soup = BeautifulSoup(response.text, 'lxml')
    text = ' '.join([e.text.split('\n') for e in soup.find_all('div', class_='petition')][0][4:-2])
    text = '. '.join(text.split('.')).replace('  ',' ')
    try:
        sign = [FQDN + e.a.get('href') for e in soup.find_all('p', class_='undersigned')][0]
    except (IndexError):
        sign = [FQDN + e.a.get('href') for e in soup.find_all('div', class_='petition')][0]
    content.append(text)
    response = get_html(sign)
    soup = BeautifulSoup(response.text, 'lxml')
    count = [str(e.strong) for e in soup.find_all('div', id = 'petition-sign')][1:2]
    count = int(''.join(re.findall('\d', count[0])))
    subscribers.append(count)

In [ ]:
df = pd.DataFrame()
df['Название'] = titles
df['Адресат'] = destination
df['Инициатор'] = source
df['Контент'] = content
df['Подписей'] = subscribers
df['Ссылка'] = fqdn_urls
df.head()

In [ ]:
file_name = FQDN.split('/')[2].split('.')[1] + '_' + str(datetime.now().date()) + '.xlsx'
with pd.ExcelWriter(file_name) as writer:
    df.to_excel(writer, sheet_name='Лист1', index=False)